In [33]:
import pandas as pd
import requests
import re
import time
pd.set_option("display.max_columns", None)

### Creamos función para llamada de api endpoint:
###### parametros comedia: {"genre": "Comedy", "startYear": "2010", "titleType": "movie", "endYear": "2024"}
###### parametros drama: {"genre":"Drama","startYear":"2010","titleType":"movie", "endYear": "2024"}
###### parametros drama: {"genre":"Action","startYear":"2010","titleType":"movie", "endYear": "2024"}

In [34]:

headers = {
    "X-RapidAPI-Key": "1db777625dmshdce6ed30bc0ec06p198b62jsn978c35c52050",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}
querystring = {"genre":"Drama","startYear":"2010","titleType":"movie", "endYear": "2024"}   #Parámetros encontrados en documentación de la API.
url = "https://moviesdatabase.p.rapidapi.com/titles"    #url endpoint.

def llamar_api(url, headers, querystring):
    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error en la solicitud:", response.status_code, response.reason)
        return None

# Llamada inicial a la API
resultados_iniciales = llamar_api(url, headers, querystring)
if resultados_iniciales:
    print(resultados_iniciales)  # Imprime los resultados iniciales para verificar
else:
    print("No se recibieron datos.")


{'page': 1, 'next': '/titles?titleType=movie&endYear=2024&startYear=2010&genre=Drama&page=2', 'entries': 10, 'results': [{'_id': '61e58ea278c012aa34446beb', 'id': 'tt10579986', 'primaryImage': {'id': 'rm878544897', 'width': 1217, 'height': 1792, 'url': 'https://m.media-amazon.com/images/M/MV5BNDFmYWRhYmMtMDBiNS00OWZkLTk3ZmYtODMyZjdlMDFjNWNlXkEyXkFqcGdeQXVyNjA1NjIzMzQ@._V1_.jpg', 'caption': {'plainText': "Mom's Coming (2024)", '__typename': 'Markdown'}, '__typename': 'Image'}, 'titleType': {'displayableProperty': {'value': {'plainText': '', '__typename': 'Markdown'}, '__typename': 'DisplayableTitleTypeProperty'}, 'text': 'Movie', 'id': 'movie', 'isSeries': False, 'isEpisode': False, 'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}], 'canHaveEpisodes': False, '__typename': 'TitleType'}, 'titleText': {'text': "Mom's Coming", '__typename': 'TitleText'}, 'originalTitleText': {'text': "Mom's Coming", '__typename': 'TitleText'}, 'releaseYear': {'year': 2024, 'endYear': Non

In [35]:
def llamada_api_paginada(url, headers, querystring):
    datos_paginados = []
    base_url = "https://moviesdatabase.p.rapidapi.com"  # url base para completar las urls de las paginas siguientes.
    while url:
        # Verifica si la url esta completa, si no, la completa con nuestros parametros. Evita de errores para paginas que no empiezan por 'http'.
        if not url.startswith('http'):
            url = base_url + url
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            json_data = response.json()
            datos_paginados.extend(json_data['results'])  #Si usaramos append() en lugar de extend(), creariamos una lista de listas, no con una lista única y plana de resultados. 
            url = json_data.get('next')  # Obtiene la siguiente página si existe
            querystring = {}  # Limpia el querystring porque la url 'next' debería incluir los parámetros necesarios.
        else:
            print("Error en la solicitud:", response.status_code, response.reason)
            return None
    return {"results": datos_paginados}



In [36]:


# Llamada completa con paginación
datos_completos = llamada_api_paginada(url, headers, querystring)
if datos_completos:
    print(datos_completos)  # Imprime los resultados para verificar la paginación
else:
    print("No se recibieron datos.")


{'results': [{'_id': '61e58ea278c012aa34446beb', 'id': 'tt10579986', 'primaryImage': {'id': 'rm878544897', 'width': 1217, 'height': 1792, 'url': 'https://m.media-amazon.com/images/M/MV5BNDFmYWRhYmMtMDBiNS00OWZkLTk3ZmYtODMyZjdlMDFjNWNlXkEyXkFqcGdeQXVyNjA1NjIzMzQ@._V1_.jpg', 'caption': {'plainText': "Mom's Coming (2024)", '__typename': 'Markdown'}, '__typename': 'Image'}, 'titleType': {'displayableProperty': {'value': {'plainText': '', '__typename': 'Markdown'}, '__typename': 'DisplayableTitleTypeProperty'}, 'text': 'Movie', 'id': 'movie', 'isSeries': False, 'isEpisode': False, 'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}], 'canHaveEpisodes': False, '__typename': 'TitleType'}, 'titleText': {'text': "Mom's Coming", '__typename': 'TitleText'}, 'originalTitleText': {'text': "Mom's Coming", '__typename': 'TitleText'}, 'releaseYear': {'year': 2024, 'endYear': None, '__typename': 'YearRange'}, 'releaseDate': {'day': 24, 'month': 11, 'year': 2024, '__typename': 'ReleaseD

#### Creamos funcion para extraer y ordenar sólo los datos que necesitamos y los retornamos en forma de DataFrame:

In [37]:
def extraer_datos_a_dataframe(data_json):   #datos_completos reemplazaría a "data_jason" en este caso. 
    datos_ordenados = []
    if data_json:  # Verifica que data_json no es None y evita errores.
        movies = data_json['results']
        for movie in movies:
            movie_id = movie['id']
            title_name = movie['titleText']['text'] if 'titleText' in movie else 'Desconocido'
            title_type = movie['titleType']['text'] if 'titleType' in movie else 'Cortometraje'   #Los que no son movies son cortometrajes.
            release_date = movie.get('releaseDate', {})     #Evita errores si no existe dia de estreno
            release_year = movie.get('releaseDate', {}).get('year', 'Desconocido') if release_date else 'Desconocido'
            release_month = movie.get('releaseDate', {}).get('month', 'Desconocido') if release_date else 'Desconocido'
            
            datos_ordenados.append({
                "ID": movie_id,
                "Nombre": title_name,
                "Tipo": title_type,
                "Año de Estreno": release_year,
                "Mes de Estreno": release_month
            })
    return pd.DataFrame(datos_ordenados)

#### Mostramos DataFrame para verificar:

In [38]:

df_completos = extraer_datos_a_dataframe(datos_completos)
print(df_completos)  


              ID                     Nombre   Tipo Año de Estreno  \
0     tt10579986               Mom's Coming  Movie           2024   
1     tt10597228        Love Song and Power  Movie           2024   
2     tt11097384                   Spaceman  Movie           2024   
3     tt11152168        If - Låtsaskompisar  Movie           2024   
4     tt11237950  The Accompanist Awakening  Movie           2024   
...          ...                        ...    ...            ...   
1093  tt32167304                    Silence  Movie           2024   
1094  tt32169685                    Vaguito  Movie           2024   
1095  tt32174039        Fenómenos naturales  Movie    Desconocido   
1096  tt32173787               Stay with Me  Movie           2021   
1097  tt32165757               Deep Thought  Movie           2017   

     Mes de Estreno  
0                11  
1                 1  
2                 3  
3                 5  
4                12  
...             ...  
1093             

##### Guardamos un '.csv' para cada género. 

In [39]:
# df_comedia.to_csv("comedia-dataframe.csv", index=False, encoding='utf-8') #utf-8' asegura que todos los caracteres se codifiquen correctamente, especialmente útil si tienes datos que pueden incluir caracteres especiales.
# df_completos.to_csv("drama-dataframe.csv", index=False, encoding='utf-8')
# df_completos.to_csv("accion-dataframe.csv", index=False, encoding='utf-8')


In [40]:
display(df_completos)

,ID,Nombre,Tipo,Año de Estreno,Mes de Estreno
0,tt10579986,Mom's Coming,Movie,2024,11
1,tt10597228,Love Song and Power,Movie,2024,1
2,tt11097384,Spaceman,Movie,2024,3
3,tt11152168,If - Låtsaskompisar,Movie,2024,5
4,tt11237950,The Accompanist Awakening,Movie,2024,12
...,...,...,...,...,...
1093,tt32167304,Silence,Movie,2024,2
1094,tt32169685,Vaguito,Movie,2024,4
1095,tt32174039,Fenómenos naturales,Movie,Desconocido,Desconocido
1096,tt32173787,Stay with Me,Movie,2021,5


#### Convertimos los DataFrame.csv a una lista de tuplas como pide el ejercicio.
##### itertuples() es un método de pandas DataFrame que itera sobre las filas del DataFrame como tuplas. A diferencia de iterrows(), que devuelve cada fila como un par (índice, Serie), itertuples() devuelve una tupla para cada fila, lo cual es generalmente más eficiente tanto en tiempo de ejecución como en uso de memoria.

In [41]:

df = pd.read_csv("drama-dataframe.csv")
lista_de_tuplas_accion = [tupla for tupla in df.itertuples(index=False, name=None)]  ##name=None sugiere que las tuplas generadas no necesitan ser nombradas.itertuples() devuelve tuplas nombradas por defecto.
print(lista_de_tuplas_accion)       #tupla representa una fila del DataFrame df, convertida en tupla.

[('tt10579986', "Mom's Coming", 'Movie', '2024', '11'), ('tt10597228', 'Love Song and Power', 'Movie', '2024', '1'), ('tt11097384', 'Spaceman', 'Movie', '2024', '3'), ('tt11152168', 'If - Låtsaskompisar', 'Movie', '2024', '5'), ('tt11237950', 'The Accompanist Awakening', 'Movie', '2024', '12'), ('tt11822244', 'Manje Bistre 3', 'Movie', '2024', '7'), ('tt12053936', 'Eternal Oath', 'Movie', '2024', '7'), ('tt12193500', 'The Passion of the Gods', 'Movie', '2024', '6'), ('tt12255500', 'Escaping Tomorrow', 'Movie', nan, nan), ('tt12767592', 'After dark', 'Movie', '2024', '12'), ('tt12861438', 'Heaven and Hell', 'Movie', '2024', '7'), ('tt13036100', 'Broken Glass', 'Movie', '2024', '2'), ('tt13433802', 'A Quiet Place: Day One', 'Movie', '2024', '6'), ('tt13561128', 'Three Couples Freed', 'Movie', '2024', '1'), ('tt13576680', 'Vo Ladki', 'Movie', '2024', '11'), ('tt1365464', 'Gina', 'Movie', '2024', '9'), ('tt13695406', "We Don't Say Retard Anymore", 'Movie', '2024', '4'), ('tt13965596', 'Rec

In [42]:
df = pd.read_csv("comedia-dataframe.csv")
lista_de_tuplas_accion = [tupla for tupla in df.itertuples(index=False, name=None)]  
print(lista_de_tuplas_accion)       

[('tt10408218', 'Pizzodyssey', 'Movie', '2024', '2'), ('tt11152168', 'If - Låtsaskompisar', 'Movie', '2024', '5'), ('tt12975102', 'Staycation', 'Movie', '2024', '4'), ('tt13036100', 'Broken Glass', 'Movie', '2024', '2'), ('tt13176330', 'Hitpig', 'Movie', '2024', '4'), ('tt13311746', 'Food Fight', 'Movie', '2024', '2'), ('tt13622970', 'Moana 2', 'Movie', '2024', '11'), ('tt13695406', "We Don't Say Retard Anymore", 'Movie', '2024', '4'), ('tt14256984', 'Una Noche Buena', 'Movie', '2024', '12'), ('tt14257662', 'MoviePass, MovieCrash', 'Movie', '2024', '3'), ('tt14478740', 'Rhapsody in Justice', 'Movie', '2024', '5'), ('tt14866430', 'Disconnected', 'Movie', '2024', '11'), ('tt15005992', 'Mookie Mayberry: Laugh, Live A Little', 'Movie', nan, nan), ('tt15009428', 'Argylle', 'Movie', '2024', '2'), ('tt15127142', 'Nothing Unusual', 'Movie', '2024', '10'), ('tt15482424', 'Untitled Universal Animated Event Film', 'Movie', '2024', '2'), ('tt15535164', 'Sunset Shimmer and the Acidic Universe of Ma

In [43]:
df = pd.read_csv("accion-dataframe.csv")
lista_de_tuplas_accion = [tupla for tupla in df.itertuples(index=False, name=None)]  
print(lista_de_tuplas_accion)   

[('tt10225380', 'Multiverso', 'Movie', '2024', '6'), ('tt10844184', 'Detour 95', 'Movie', '2024', '10'), ('tt10928904', 'The Legend of Johnny Jones', 'Movie', '2024', '10'), ('tt11078340', 'Slasher House 3', 'Movie', '2024', '3'), ('tt11374514', 'Clank and the Golden Scar', 'Movie', '2024', '7'), ('tt12037194', 'Furiosa: A Mad Max Saga', 'Movie', '2024', '5'), ('tt12193500', 'The Passion of the Gods', 'Movie', '2024', '6'), ('tt12529250', 'Danya', 'Movie', nan, nan), ('tt13143064', 'Adrienne', 'Movie', nan, nan), ('tt13176330', 'Hitpig', 'Movie', '2024', '4'), ('tt13542474', '1242: Gateway to the West', 'Movie', '2024', '9'), ('tt13590374', 'Mnemonix', 'Movie', '2024', '12'), ('tt13818368', 'Fighter', 'Movie', '2024', '1'), ('tt14114688', "L'Inferno (Remastered)", 'Movie', '2024', '9'), ('tt14564000', 'Guntur Kaaram', 'Movie', '2024', '1'), ('tt15009428', 'Argylle', 'Movie', '2024', '2'), ('tt15127142', 'Nothing Unusual', 'Movie', '2024', '10'), ('tt15239678', 'Kopa. Antra dalis', 'Mov